In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn import utils
import wandb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import stats

pd.set_option('display.max_columns', 500)

# local modules
from utils.config import Config, Config_AE
import utils.utils as utils
import utils.plots as plots
import modelsNN.modelsNN as modelsNN

In [2]:
# Local:
PATH_DEEPSF = 'C:/Users/joseb/Downloads/deepsf/data'

In [3]:
# %%% Setting deepAE NN characteristics.
configAE = Config_AE()
configAE.get_config()  
configSF = Config()
configSF.get_config()

{'batch_size': 256,
 'optimizer': 'Adam',
 'epochs': 40000,
 'learning_rate': 0.0001,
 'if_wandb': True,
 'test_size': 0.2,
 'tumor_type': ['LUAD']}

{'batch_size': 32,
 'optimizer': 'adamW',
 'modelNN': 'DeepSF_2hidden',
 'epochs': 3000,
 'learning_rate': 0.0001,
 'if_toy': False,
 'if_wandb': False,
 'test_size': 0.2,
 'num_genes': 898,
 'tumor_type': ['LUAD']}

In [4]:
# %%% Get the data
data_AE = utils.get_data_AE(PATH_DEEPSF, configAE)
data_prep = utils.get_data(PATH_DEEPSF, configSF)

In [5]:
data_ensemble = utils.get_model_loader_ensemble('model_DeepAE_adamW.pt', PATH_DEEPSF, data_prep, data_AE, configSF)

model = data_ensemble.model
train_loader = data_ensemble.train_loader
val_loader = data_ensemble.val_loader

model

DeepSF_AE_Ensemble(
  (modelAE): Sequential(
    (0): Linear(in_features=19594, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
  )
  (linear1): Linear(in_features=1279, out_features=183, bias=True)
  (bn1): BatchNorm1d(183, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=183, out_features=82, bias=True)
  (bn2): BatchNorm1d(82, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=594, out_features=10095, bias=True)
)

In [6]:
optimizer = utils.build_optimizer(model, configSF.optimizer, configSF.learning_rate)
history = utils.fit(10, 
                    model, 
                    train_loader, 
                    val_loader, 
                    optimizer=optimizer,
                    hyperparameters = configSF.get_config(),
                    if_wandb = configSF.if_wandb,
                    path = PATH_DEEPSF,
                    model_name = 'prueba2.pt',
                    project_name = 'DeepSF_AE_Ensemble_training')

# %%% Plot results
solution = plots.plot_results(history = history, 
                     scaledTrain_df = data_prep.scaledTrain_df, 
                     train_labels = data_prep.train_labels, 
                     scaled_train_gn = data_prep.scaled_train_gn, 
                     scaledValidation_df = data_prep.scaledValidation_df, 
                     valid_labels = data_prep.valid_labels , 
                     scaled_valid_gn = data_prep.scaled_valid_gn, 
                     model = model, 
                     getBM = data_prep.getBM,
                     if_wandb = configSF.if_wandb,
                     config = configSF,
                     if_ensemble=True, 
                     xa_tr = data_AE.TCGA_tpm_gn_prot_cod.loc[data_prep.scaledTrain_df.index],
                     xa_val = data_AE.TCGA_tpm_gn_prot_cod.loc[data_prep.scaledValidation_df.index])
        
if configSF.if_wandb:
    wandb.finish()

solution.solution_train_cor_total
solution.solution_train_df
solution.solution_val_cor_total
solution.solution_val_df

Epoch [0], loss: 1455.6373, val_loss: 1106.6455
Epoch [1], loss: 890.6907, val_loss: 746.3263
Epoch [2], loss: 603.6586, val_loss: 531.6796
Epoch [3], loss: 439.9738, val_loss: 386.2728
Epoch [4], loss: 317.7553, val_loss: 283.8972
Epoch [5], loss: 237.5688, val_loss: 211.7091
Epoch [6], loss: 176.8582, val_loss: 160.0069
Epoch [7], loss: 135.1878, val_loss: 122.1896
Epoch [8], loss: 101.7263, val_loss: 94.9110
Epoch [9], loss: 80.1493, val_loss: 74.7522
Model saved succesfully in C:/Users/joseb/Downloads/deepsf/data


AttributeError: 'DataAE' object has no attribute 'TCGA_tpm_gn_prot_cod'